

### Key Changes:

1. **Google Gemini as Chat and Embedding Model**:
   - The embedding model is set as `GoogleGenerativeAIEmbeddings` with `embedding-001`.
   - The chat model used for the conversational chain is `ChatGoogleGenerativeAI`, with the model set to `"gemini-pro"`.

2. **FAISS Vector Store**:
   - FAISS is used to store and retrieve embeddings from the text chunks created from the PDF. 
   - The FAISS index is saved locally (`faiss_index`), and is loaded back when generating responses.

3. **Streamlit App**:
   - The app is designed using Streamlit, allowing users to input questions, and based on those questions, the system performs a similarity search using the FAISS index to retrieve relevant chunks from the PDF.
   - A response is then generated using the Google Gemini chat model.

4. **Error Handling**:
   - Added checks for creating the FAISS index only once. If the FAISS index already exists, it skips reprocessing the PDF.

### Explanation:

- **PDF Processing**: The PDF file is processed once, splitting it into chunks using `RecursiveCharacterTextSplitter`. Embeddings are generated for each chunk, and they are stored in a FAISS vector store.
- **Query Handling**: When the user submits a question, the system searches for the most relevant chunks in the FAISS index using similarity search and sends the relevant chunks to the conversational chain.
- **Generative Model**: The `ChatGoogleGenerativeAI` model (`gemini-pro`) is used to generate a response based on the user's query and the context retrieved from the PDF.

This should work well with your Google API key and the Gemini model. Let me know if you encounter any issues or need further adjustments!

In the line:

```python
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
```

You are creating a **question-answering (QA) chain** using Langchain, a framework that helps link large language models (LLMs) with data sources, tools, and custom logic to build advanced applications like chatbots or QA systems.

Let's break down this line to understand each part:

### 1. **`load_qa_chain(model, chain_type="stuff", prompt=prompt)`**
This function, `load_qa_chain`, loads a **question-answering chain** in Langchain. It defines how the system will handle the process of answering questions based on context and a user-provided query.

- **`model`**: This is the large language model (LLM) that will be used to generate answers. In your case, the model is `ChatGoogleGenerativeAI`, which interacts with Google's Gemini AI to provide conversational responses.
  
- **`chain_type="stuff"`**: This specifies the **type of chain** being used. Langchain provides different types of QA chains that handle how the model processes the context and questions:
  
  - **"stuff"**: This is the simplest chain type. It essentially takes all the retrieved or provided documents (context) and "stuffs" them together as input to the model, along with the user's question. The model then generates an answer based on that combined input. It's a straightforward method and works well for smaller contexts that can fit into a single model call.
  
  - Other chain types, like `"map_reduce"` or `"refine"`, handle more complex workflows by breaking down the context or using multiple steps to refine the answer. However, `"stuff"` is quick and efficient for relatively small or concise inputs.
  
- **`prompt=prompt`**: This is the **custom prompt template** you're providing to the model. The `PromptTemplate` object defines how the context and question should be formatted when passed to the model. In your case, the prompt includes specific instructions on how to answer the question based on the context, and what to do if the answer is not found in the context (e.g., return "answer is not available in the context").

### 2. **`chain`**
The `chain` is the result of this function call. It is an object that can take user questions, along with a set of documents (context), and then interact with the large language model (Gemini AI, in this case) to generate a response. The chain ensures that the model processes both the context and the user's question together to produce a relevant answer.

### Putting It All Together
The `load_qa_chain` function builds the logic for answering a question using the context provided. Here's how it works:

1. **Context**: You retrieve relevant documents or text chunks (e.g., sections of the PDF) based on similarity to the user question.
2. **Question**: The user provides a question.
3. **Prompt**: The context and the question are passed into a prompt template that guides the model on how to respond.
4. **LLM Call**: The language model (Google Gemini in this case) processes the combined context and question to generate an answer.
5. **Chain Execution**: This whole process is abstracted into a "chain", allowing you to simply pass in the context and question, and get an answer back.

### Example Flow:
1. User asks a question: "What is energy-saving tip for lighting?"
2. Relevant text is retrieved from the PDF (chunks).
3. This text, along with the user question, is passed through the `PromptTemplate`, which structures the query.
4. The structured query is sent to the language model (`ChatGoogleGenerativeAI`).
5. The model generates an answer based on the combined context and question.

The `"stuff"` chain type handles simple use cases where the context is relatively small or can be processed in one step. More complex workflows may require additional logic (like splitting the context, refining the answer, etc.), which other chain types handle.



In [7]:
from PyPDF2 import PdfReader
import os

In [8]:
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return  text

In [14]:
path = r"C:\Users\prashantvi\Desktop\POC_GOOGLE_GEMINI_PRO\chatmultipledocuments\DNDi-Clinical-Trial-Protocol-BENDITA-V5.pdf"

In [15]:

get_pdf_text(path)

FileNotFoundError: [Errno 2] No such file or directory: 'C'

In [17]:
import faiss

# Load the FAISS index
index = faiss.read_index(r"C:\Users\prashantvi\Desktop\POC_GOOGLE_GEMINI_PRO\chatmultipledocuments\faiss_index\index.faiss")

In [18]:
# Get the number of vectors in the index
num_vectors = index.ntotal
print(f"Number of vectors in index: {num_vectors}")

# Get the dimensionality of vectors
dim = index.d
print(f"Dimensionality of vectors: {dim}")

Number of vectors in index: 22
Dimensionality of vectors: 768


In [19]:
# Get a specific vector by its index
vector_id = 0  # Example: Get the first vector
vector = index.reconstruct(vector_id)
print(f"Vector at position {vector_id}: {vector}")

Vector at position 0: [ 3.39184031e-02 -8.25485215e-03 -7.05045611e-02  1.39361462e-02
  5.67322969e-02  4.24892455e-02  2.13304888e-02 -2.93323658e-02
 -5.61286835e-03  4.86250184e-02 -7.58128893e-03 -1.05367712e-04
  2.07014475e-03 -1.76353343e-02 -2.64348788e-03  9.35550593e-03
 -1.85206700e-02 -2.08219029e-02 -4.87591177e-02 -4.88178395e-02
 -3.38023575e-03 -6.63700327e-03  1.74942929e-02 -9.17675381e-04
 -1.12789441e-02 -2.92808618e-02  3.76225039e-02 -8.41830075e-02
  7.34677445e-03  4.15583439e-02 -2.12314818e-02  6.74320683e-02
 -5.15615903e-02 -1.45345163e-02 -3.03242020e-02 -4.00836281e-02
 -6.94816234e-03 -1.52827529e-02 -2.29486730e-02  1.75605901e-02
  1.25475368e-02  9.64852516e-03  1.88839417e-02  1.01014050e-02
  4.67555895e-02  2.59510763e-02  4.54778969e-03  3.52745540e-02
 -1.26099456e-02 -4.36802767e-02 -1.68989860e-02 -6.66722702e-03
  3.91951129e-02  2.10799742e-02  2.64200345e-02 -4.00602771e-03
  2.91903447e-02 -2.06581000e-02 -1.36046046e-02  2.04537082e-02
  2